# Regresión Logística con Base de datos de Mercadeo de Bancos

## Importar las librerías

In [1]:
import pandas as pd
import numpy as np

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import r2_score

## Ordenar los datos

In [2]:
df = pd.read_csv("bank.csv", sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


#### Convertir los 'yes'/'no' en valores numéricos (0 y 1)

In [3]:
df['y'] = [0 if x == 'no' else 1 for x in df['y']] #Reemplazamos los 'yes' 'no' por valores '0' y '1'

X = df.drop('y', 1) #dejamos la columna de caracteres
Y = df.y  #elegimos la columna de los resultados en números

Y.value_counts()

0    4000
1     521
Name: y, dtype: int64

* Hay 4000 'no'   y   521 'yes'

## Se analizan las columnas con categorías y se vuelven numéricas

In [4]:
catego = []
for col_name in X.columns:
    if X[col_name].dtypes == 'object':# in pandas it is object
        catego.append(col_name) #Se añaden las categorías a la lista
        unique_cat = len(X[col_name].unique())
        print("El feature '{col_name}' tiene {unique_cat} categorías únicas".format(col_name=col_name, unique_cat=unique_cat))

El feature 'job' tiene 12 categorías únicas
El feature 'marital' tiene 3 categorías únicas
El feature 'education' tiene 4 categorías únicas
El feature 'default' tiene 2 categorías únicas
El feature 'housing' tiene 2 categorías únicas
El feature 'loan' tiene 2 categorías únicas
El feature 'contact' tiene 3 categorías únicas
El feature 'month' tiene 12 categorías únicas
El feature 'poutcome' tiene 4 categorías únicas


* Para volver una columna categórica en numérica se crean 'variables dummies'
* Estas representan cada una de las categorías y se asignan con '0' ó '1'

In [5]:
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False) # prefix give name
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [6]:
X = dummy_df(X, catego)
catego_dummies = list(X.columns)

In [7]:
X.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1


## Se divide el dataset en training y testing set

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

## Se intenta con un modelo de clasificación de SVM

![Clasificador SVM](https://scikit-learn.org/stable/_images/sphx_glr_plot_separating_hyperplane_unbalanced_0011.png)

In [9]:
cls = svm.SVC().fit(X_train, Y_train)

In [10]:
preds_train = cls.predict(X_train) # Se predicen los valores los datos de entrenamiento
print("Accuracy:",metrics.accuracy_score(Y_train, preds_train)) #Se comparan con los resultados reales

Accuracy: 0.8864306784660767


In [11]:
preds_test = cls.predict(X_test) # Se predicen los valores los datos de entrenamiento
print("Accuracy:",metrics.accuracy_score(Y_test, preds_test)) #Se comparan con los resultados reales

Accuracy: 0.8822553897180763


## Ensayamos un modelo de regresión logística (categorías) de Sklearn

In [12]:
clf = LogisticRegression(max_iter=10000).fit(X_train, Y_train) #Se crea el modelo y se ajusta a los valores de entrenamiento

In [13]:
preds_train = clf.predict(X_train) # Se predicen los valores los datos de entrenamiento
print("Accuracy:",metrics.accuracy_score(Y_train, preds_train)) #Se comparan con los resultados reales

Accuracy: 0.9111356932153393


In [14]:
preds_test = clf.predict(X_test) # Se predicen los datos de testeo
print("Accuracy:",metrics.accuracy_score(Y_test, preds_test)) # Se compara con los resultados reales

Accuracy: 0.8899944720840243


## Usando el modelo de Clasificador de Bosque Aleatorio

In [15]:
forest = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=100)
forest.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
                       oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

#### La precisión del Training set es de 1.0

In [16]:
preds_train = forest.predict(X_train)
r2_score(Y_train, preds_train)

print("Accuracy:",metrics.accuracy_score(Y_train, preds_train))

Accuracy: 1.0


#### La precisión para el test set es de 0.89

In [17]:
preds_test = forest.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, preds_test))

Accuracy: 0.8977335544499724


### Ahora analicemos

In [18]:
#Obtenemos la importancia de cada feature y lo redondeamos a decimal con 2 dígitos después del 0
features = forest.feature_importances_
features = [round(i,2) for i in features]

In [19]:
#La importancia de los parámetros en orden 
features.sort(reverse=True)
features[0:5]

[0.25, 0.09, 0.08, 0.08, 0.04]

In [20]:
list(zip(catego_dummies, features))

[('age', 0.25),
 ('balance', 0.09),
 ('day', 0.08),
 ('duration', 0.08),
 ('campaign', 0.04),
 ('pdays', 0.04),
 ('previous', 0.04),
 ('job_admin.', 0.03),
 ('job_blue-collar', 0.02),
 ('job_entrepreneur', 0.01),
 ('job_housemaid', 0.01),
 ('job_management', 0.01),
 ('job_retired', 0.01),
 ('job_self-employed', 0.01),
 ('job_services', 0.01),
 ('job_student', 0.01),
 ('job_technician', 0.01),
 ('job_unemployed', 0.01),
 ('job_unknown', 0.01),
 ('marital_divorced', 0.01),
 ('marital_married', 0.01),
 ('marital_single', 0.01),
 ('education_primary', 0.01),
 ('education_secondary', 0.01),
 ('education_tertiary', 0.01),
 ('education_unknown', 0.01),
 ('default_no', 0.01),
 ('default_yes', 0.01),
 ('housing_no', 0.01),
 ('housing_yes', 0.01),
 ('loan_no', 0.01),
 ('loan_yes', 0.01),
 ('contact_cellular', 0.01),
 ('contact_telephone', 0.01),
 ('contact_unknown', 0.01),
 ('month_apr', 0.01),
 ('month_aug', 0.01),
 ('month_dec', 0.01),
 ('month_feb', 0.01),
 ('month_jan', 0.0),
 ('month_jul', 